In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from autogen.agentchat import AssistantAgent
from autogen.interop import (
    Interoperability,  # noqa: F401
    Interoperable,
    MCPInteroperability,
)
from autogen.tools import Toolkit  # noqa: F401

In [ ]:
interop: Interoperable = MCPInteroperability()

# runtime check
assert isinstance(interop, Interoperable)

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your math_server.py file
    args=["math_server.py"],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # mcp_tools = await session.list_tools()
        # interop = Interoperability()
        # tools = [interop.convert_tool(tool=mcp_tool, type="mcp", session=session) for mcp_tool in mcp_tools.tools]
        # toolkit = Toolkit(tools=tools)

        toolkit = await MCPInteroperability.load_mcp_toolkit(session=session)

        agent = AssistantAgent(name="assistant", llm_config={"model": "gpt-4o-mini", "api_type": "openai"})
        toolkit.register_for_llm(agent)

        agent.run(
            message="Add 123223 and 456789",
            tools=toolkit.tools,
            max_turns=2,
            user_input=False,
        )